# นนทกร มาคีรี 60070144 assignment 2

## Web scraping

In [1]:
import requests
from bs4 import BeautifulSoup
page = requests.get("http://www.it.kmitl.ac.th/~teerapong/news_archive/index.html")
soup = BeautifulSoup(page.text, 'html.parser')

In [2]:
lst_month = []
for i in soup.find_all('a',href=True):  #เก็บ link แต่ละเดือนเข้า list
    req_month = "http://www.it.kmitl.ac.th/~teerapong/news_archive/"+ i['href']
    lst_month.append(req_month)

In [3]:
lst_month

['http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jan-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-feb-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-mar-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-apr-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-may-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jun-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jul-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-aug-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-sep-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-oct-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-nov-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/month-dec-2017.html',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/#',
 'http://www.it.kmitl.ac.th/~teerapong/news_archive/#',


วน loop แต่ละเดือนเพื่อเก็บค่า Category

In [4]:
lsttype = [] #สร้าง list มาเก็บค่า category
for type_ in lst_month[0:12]: #เอาแค่ 12 item ในลิส(12 เดือน)
    page_type = requests.get(type_)   # request แต่ละเดือน
    soup_type = BeautifulSoup(page_type.text, 'html.parser')
    for addtype in soup_type.find_all('td',{"class": "category"}): #loop ใน tag ที่เป็น category
        lsttype.append(addtype.get_text().lstrip('\xa0')) # strip ตัดคำ '\xa0sport' ให้เหลือแค่ 'sport'

In [5]:
for i in lsttype:
    if i == 'N/A':
        lsttype.remove(i) # ตัดค่า N/A ออก
file = open('datastore/category.txt','w',encoding='utf-8') #เรียกไฟล์ที่จะเก็บคือ category.txt
for i in lsttype:
    file.write(i+'\n') #เขียน item ที่เป็น category ลงในไฟ category.txt และให้ขึ้นบรรทัดใหม่เมื่อจบ 1 loop
file.close()
len(lsttype) #ดูจำนวน category

1408

In [6]:
lstitem = [] #สร้าง list มาเพื่อเก็บ link ของแต่ละข่าว
for i in lst_month[0:12]:
    pages = requests.get(i)  # request แต่ละเดือน
    soups = BeautifulSoup(pages.text, 'html.parser')
    item = soups.find('tbody') 
    for j in item.find_all('a', href=True):  #วนลูปเก็บ link ข่าวทุกข่าว
        req_item = "http://www.it.kmitl.ac.th/~teerapong/news_archive/"+ j['href']
        lstitem.append(req_item)

In [7]:
import time
lstcontent = [] #สร้าง list เพื่อมาเก็บ text ข่าวทั้งหมด
for i in lstitem: # วน loop แต่ละ link เพื่อเก็บตัว Body ของทุกข่าว
    time.sleep(0.1)
    page_content = requests.get(i) # request แต่ละ link ของข่าว
    soup_content = BeautifulSoup(page_content.content, 'html.parser')
    content = soup_content.find_all('p') #เอา tag p มาทั้งหมด
    tmp = []
    for j in content[1:-1:1]: #วน loop ใน tag p แต่ไม่เอาตัวแรกและตัวสุดท้าย
        tmp.append(j.get_text()) #แปลง tag p ให้เป็น text
    lstcontent.append(''.join(tmp)) #เนื่องจากใน 1 ข่าวมีหลาย tag p เลยใช้ .join เพื่อเอา text มาต่อกัน

In [8]:
file = open('datastore/news.txt','w',encoding="utf-8") #เรียกไฟล์ที่จะเก็บคือ news.txt
for i in lstcontent:
    file.writelines(i+'\n') #เขียน item ที่เป็น text ของข่าว ลงในไฟ news.txt และให้ขึ้นบรรทัดใหม่เมื่อจบ 1 loop
file.close()
len(lstcontent) #ดูจำนวนข่าวทั้งหมด

1408

# text classification

เรียก 2 ไฟล์คือ category และ news เพื่อเก็บเป็น X,Y

In [9]:
file = open("datastore/news.txt",'r',encoding="utf8")
file_read = file.readlines() 
file.close()
file_news = []
for i in file_read:
    file_news.append(i.rstrip('\n')) #ตัด \n ออก
file_news[0] #ค้นหาเพื่อดูข่าวแรก

'The sporting industry has come a long way since the ‘60s. It has carved out for itself a niche with its roots so deep that I cannot fathom the sports industry showing any sign of decline any time soon - or later.The reason can be found in this seemingly subtle difference - other industries have customers; the sporting industry has fans. Vivek Ranadivé, leader of the ownership group of the NBA’s Sacramento Kings, explained it beautifully, “Fans will paint their face purple, fans will evangelize. ... Every other CEO in every business is dying to be in our position — they’re dying to have fans.“While fan passion alone could almost certainly keep the industry going, leagues and sporting franchises have decided not to rest on their laurels. The last few years have seen the steady introduction of technology into the world of sports - amplifying fans’ appreciation of games, enhancing athletes’ public profiles and informing their training methods, even influencing how contests are waged.Also,

In [10]:
file2 = open("datastore/category.txt",'r',encoding="utf8")
file_type = file2.readlines()
file2.close()
Y = []
for i in file_type:
    Y.append(i.rstrip("\n"))

# Text preprocessing

ใช้ Tfid ทำการถ่วงน้ำหนักและ stop word ตัดคำที่ไม่จำเป็นออก

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english",min_df = 5)
X = vectorizer.fit_transform(file_news)

# สร้าง Model (naive bayes, decisiontree, knn)
### ใช้ cross validation 10 fold

In [12]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score #import library ที่ใช้ให้การทำ cross validation
model_nb = BernoulliNB(alpha=0.2)
cv_naive = cross_val_score(model_nb, X, Y, cv=10)
print(cv_naive)                                                         #ค่า accuracy ทั้ง 10 กลุ่มของ naive_bayes
print("Accuracy naive bayes in cross validation : %0.3f (+/- %0.3f)" % (cv_naive.mean(), cv_naive.std())) #แสดงค่าเฉลี่ยของ accuracy และค่า standard deviation ของ naive bayes

[0.97902098 0.9858156  0.9858156  0.9929078  0.9787234  0.9858156
 0.99285714 0.97142857 0.97142857 0.95714286]
Accuracy naive bayes in cross validation : 0.980 (+/- 0.011)


In [13]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=1) 
cv_tree = cross_val_score(decision_tree, X, Y, cv=10)
print(cv_tree)                                                                                    #ค่า accuracy ทั้ง 10 กลุ่มของ decistion tree
print("Accuracy decision tree in cross validation: %0.3f (+/- %0.3f)" % (cv_tree.mean(), cv_tree.std())) #แสดงค่าเฉลี่ยของ accuracy และค่า standard deviation ของ decistion tree

[0.8951049  0.86524823 0.89361702 0.91489362 0.87943262 0.86524823
 0.93571429 0.89285714 0.9        0.87857143]
Accuracy decision tree in cross validation: 0.892 (+/- 0.021)


In [14]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=15)
cv_knn = cross_val_score(neigh, X, Y, cv=10)
print(cv_knn)                                                                                    #ค่า accuracy ทั้ง 10 กลุ่มของ knn
print("Accuracy knn in cross validation: %0.3f (+/- %0.3f)" % (cv_knn.mean(), cv_knn.std())) #แสดงค่าเฉลี่ยของ accuracy และค่า standard deviation ของ knn

[0.97902098 0.97163121 0.9787234  0.97163121 0.95035461 0.96453901
 0.96428571 0.96428571 0.98571429 0.94285714]
Accuracy knn in cross validation: 0.967 (+/- 0.012)


## สรุปผล

ผลลัพธ์ที่ได้จาก 3 model ข้างบน ทำให้สรุปผลได้ว่า โมเดล Naive Bayes ได้ค่า Acuuracy เฉลี่ยเท่ากับ 0.980 +/- 0.011 ซึ่งสูงสุดเมื่อเทียบกับอีก 2 โมเดล
ทำให้บอกได้ว่าโมเดล Naive Bayes มีประสิทธิภาพในการจำแนกหมวดหมู่ข่าวจากบทความข่าวได้ดีที่สุด